In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [6]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [7]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [8]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [9]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [12]:
##########################################
# --- Agente 1: Mapeador de Recursos --- #
##########################################
def agente_mapeamento_recursos():
    mapeador = Agent(
        name="agente_mapeamento_recursos",
        model="gemini-2.0-flash", # Ou um modelo mais robusto se precisar de mais raciocínio
        instruction="""
        Você é um Agente de Mapeamento e Recursos. Sua tarefa é coletar, verificar e manter atualizadas informações
        sobre abrigos, locais de alimentação (refeitórios, sopas), postos de saúde, centros de assistência social (CRAS/CREAS),
        locais de banho público, lavanderias comunitárias e oportunidades de emprego para pessoas em situação de rua.
        Utilize a ferramenta Google Search (@google_search) para encontrar informações em sites de prefeituras, ONGs e outras organizações.
        Seja meticuloso para garantir a precisão dos dados (endereço, horário, requisitos de acesso, contato, vagas se possível).
        Retorne as informações de forma estruturada, listando o nome do recurso, tipo, endereço, horário e contato.
        Neste formato:
        [
          {
            "nome": "Abrigo Esperança",
            "tipo": "abrigo",
            "endereço": "Rua X, 123 - Bairro Y",
            "horário": "24h",
            "contato": "(11) 99999-0000"
          },
          ...
        ]
        """,
        description="Agente que mapeia e atualiza recursos para pessoas em situação de rua.",
        tools=[google_search] # Pode ser necessário adicionar ferramentas para interagir com um DB no futuro
    )
    # A entrada para este agente pode ser 'Buscar abrigos em [localidade]' ou 'Atualizar dados de [tipo de recurso]'
    # A saída deve ser um formato estruturado (lista de dicionários, JSON)
    # return call_agent(mapeador, "Busque os 5 principais abrigos em São Paulo com vagas.")
    return mapeador # Retorna o objeto do agente para ser usado em outras chamadas


In [15]:
################################################
# --- Agente 2: Interação de Necessidades --- #
################################################
def agente_interacao_necessidades():
    interacao = Agent(
        name="agente_interacao_necessidades",
        model="gemini-2.0-flash",
        instruction="""
        Você é um Agente de Interação e Necessidades para pessoas em situação de rua. Sua função é acolher o usuário e perguntar sobre sua necessidade imediata (ex: 'Preciso de abrigo', 'Estou com fome', 'Estou doente'). Faça perguntas claras e diretas para entender a situação:
        - Você está sozinho(a)?
        - Qual sua localização atual?
        - Precisa de algo específico?
        Ao final da conversa, retorne os dados estruturados no seguinte formato:
        {
          "necessidade": "descrição curta da necessidade",
          "localizacao": "localização fornecida ou 'não informada'"
        }
        """,
        description="Agente que interage com o usuário para entender suas necessidades.",
        # Não necessariamente precisa de ferramentas aqui, a menos que ele precise de busca simples para entender termos.
    )
    # A entrada para este agente será a mensagem direta do usuário (ex: "Estou com muito frio e não tenho onde dormir")
    # return call_agent(interacao, "Estou com muito frio e não tenho onde dormir.")
    return interacao

In [16]:
######################################
# --- Agente 3: Recomendador/Encaminhamento --- #
######################################
def agente_recomendacao_encaminhamento():
    recomendador = Agent(
        name="agente_recomendacao_encaminhamento",
        model="gemini-2.5-pro-preview-03-25", # Pode precisar de um modelo mais potente para raciocínio complexo
        instruction="""
        Você é um Agente de Recomendação e Encaminhamento. Sua função é receber uma necessidade identificada de um usuário (ex: tipo de recurso
        e localização) e consultar o Agente de Mapeamento e Recursos (AMR) para encontrar as melhores opções disponíveis.
        Priorize recomendações por:
        - Proximidade geográfica com o usuário
        - Relevância da necessidade
        - Condições específicas como disponibilidade (ex: vagas)
        Retorne uma lista de até 3 recursos no seguinte formato:
        [
          {
            "nome": "Nome do local",
            "tipo": "abrigo / refeição / posto de saúde / etc.",
            "endereco": "Endereço completo",
            "horario": "Horário de funcionamento",
            "rota": "Como chegar a pé ou de ônibus, se possível",
            "telefone": "Telefone de contato"
          }
        ]
        Se nenhuma opção for encontrada, responda com: "No momento, não encontramos recursos disponíveis para essa necessidade na região informada.
        """,
        description="Agente que recomenda recursos com base nas necessidades do usuário e dados mapeados.",
        # Ele precisará de uma forma de "chamar" ou "consultar" o AMR.
        # Isso pode ser feito integrando a função do AMR aqui ou passando a informação.
        # Por simplicidade inicial, a informação do AMR pode ser passada como parte da entrada.
        tools=[google_search] # Pode ser útil para rotas ou detalhes adicionais
    )
    # A entrada para este agente seria algo como:
    # entrada = f"Necessidade do usuário: {necessidade_do_ain}\nRecursos disponíveis: {dados_do_amr}"
    # return call_agent(recomendador, entrada)
    return recomendador

In [17]:
##########################################
# --- Agente 4: Alertas e Clima --- #
##########################################
def agente_alertas_clima():
    alertas = Agent(
        name="agente_alertas_clima",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
        Você é um Agente de Alertas e Clima.

        Sua função é monitorar proativamente:
        - Condições climáticas severas (frio/calor extremos, chuvas intensas)
        - Alertas de saúde pública (vacinações, surtos)
        - Eventos emergenciais (distribuição de alimentos, cobertores, mutirões)

        Quando alguma dessas condições for detectada, gere um alerta claro e conciso com:
        - Tipo de alerta
        - Descrição do evento
        - Data e região afetada
        - Ação recomendada (ex: "Procure abrigo", "Vacinação disponível no CRAS X")

        Retorne o alerta em formato estruturado:
        {
          "tipo_alerta": "Frio extremo",
          "descricao": "...",
          "recomendacao": "...",
          "data_hora": "...",
          "regiao_afetada": "..."
        }
        Se nenhuma condição crítica estiver presente, apenas diga: "Sem alertas no momento.
        """,
        description="Agente que monitora e emite alertas proativos sobre clima e eventos.",
        tools=[google_search] # Para buscar informações de clima/notícias de saúde pública
        # Poderia integrar APIs de clima diretamente aqui.
    )
    # A entrada para este agente poderia ser uma trigger baseada em tempo (a cada X horas)
    # return call_agent(alertas, f"Verifique alertas para hoje ({date.today().strftime('%d/%m/%Y')}).")
    return alertas

In [ ]:
import json
from datetime import date

data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Apoio a Pessoas em Situação de Rua com 4 Agentes 🚀")

# --- Instancia os Agentes ---
amr = agente_mapeamento_recursos()
ain = agente_interacao_necessidades()
are = agente_recomendacao_encaminhamento()
aac = agente_alertas_clima()

# --- Interação com o Usuário ---
necessidade_usuario_raw = input("❓ Olá! Em que posso te ajudar hoje? ")

if not necessidade_usuario_raw:
    print("⚠️ Por favor, digite sua necessidade.")
else:
    print(f"Entendido! Você precisa de ajuda com: {necessidade_usuario_raw}")

    # --- 1. AIN: Interpreta a necessidade ---
    print("\n--- 🗣️ Agente de Interação e Necessidades (AIN) ---")
    try:
        resposta_ain = call_agent(ain, necessidade_usuario_raw + " Por favor, responda em JSON.")
        necessidade_identificada = json.loads(resposta_ain)
        display(to_markdown(resposta_ain))
    except json.JSONDecodeError:
        print("❌ Erro ao interpretar a resposta do AIN.")
        necessidade_identificada = {}

    # --- 2. AMR: Busca recursos disponíveis ---
    print("\n--- 🗺️ Agente de Mapeamento e Recursos (AMR) ---")
    try:
        resposta_amr = call_agent(amr, "Liste os principais recursos de abrigo e alimentação. Retorne em JSON.")
        recursos_gerais = json.loads(resposta_amr)
        display(to_markdown(resposta_amr))
    except json.JSONDecodeError:
        print("❌ Erro ao interpretar a resposta do AMR.")
        recursos_gerais = []

    # --- 3. AAC: Verifica alertas ---
    print("\n--- 🚨 Agente de Alertas e Clima (AAC) ---")
    try:
        resposta_aac = call_agent(aac, f"Verifique alertas para {data_de_hoje}. Retorne em JSON.")
        alerta_atual = json.loads(resposta_aac)
        display(to_markdown(resposta_aac))
    except json.JSONDecodeError:
        print("❌ Erro ao interpretar a resposta do AAC.")
        alerta_atual = {}

    # --- 4. ARE: Recomendação final ---
    print("\n--- 🎯 Agente de Recomendação e Encaminhamento (ARE) ---")
    entrada_para_are = {
        "necessidade": necessidade_identificada,
        "recursos": recursos_gerais,
        "alertas": alerta_atual
    }

    try:
        resposta_are = call_agent(are, json.dumps(entrada_para_are))
        recomendacoes_finais = json.loads(resposta_are)
        display(to_markdown(resposta_are))
    except json.JSONDecodeError:
        print("❌ Erro ao interpretar a resposta do ARE.")
        recomendacoes_finais = {}

    print("\n✅ Processo de recomendação finalizado!")


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))